In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim

In [3]:
from mymodule.data_loader import get_loader
from mymodule.trainer import Trainer

### Model

In [4]:
class SequenceClassifier(nn.Module):
    
    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        n_layers=4,
        dropout_p=.2
    ):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()
        
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout_p,
            bidirectional=True
        )
        self.layers = nn.Sequential(
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size * 2),
            nn.Linear(hidden_size * 2, output_size),
            nn.Softmax(dim=-1),
        )
        
    def forward(self, x):
        # hidden_state, close_state
        z, _ = self.rnn(x)
        z = z[:, -1]
        y = self.layers(z)
        
        return y

### Data Load

In [5]:
def load_mnist(is_train=True, flatten=True):
    from torchvision import datasets, transforms

    dataset = datasets.MNIST(
        '../data', train=is_train, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
        ]),
    )

    x = dataset.data.float() / 255.
    y = dataset.targets

    if flatten:
        x = x.view(x.size(0), -1)

    return x, y

In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
        super().__init__()
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        x = self.data[i]
        y = self.labels[i]
        
        return x, y

In [7]:
device = torch.device('cuda:0')

In [8]:
x, y = load_mnist(flatten=False)

C:\src\envs\torch\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\src\envs\torch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [13]:
train_loader, valid_loader = get_loader(x, y, CustomDataset)

In [14]:
model = SequenceClassifier(28, 128, 10).to(device)

In [15]:
optimizer = optim.Adam(model.parameters())
crit = nn.CrossEntropyLoss()

In [16]:
trainer = Trainer(model, optimizer, crit)

In [17]:
trainer.train(train_loader, valid_loader, 20)

188it [00:06, 27.34it/s, Epoch=1, Loss=1.591791]
47it [00:00, 90.99it/s, Epoch=1, Val Loss=1.592154]
188it [00:06, 28.25it/s, Epoch=2, Loss=1.507258]
47it [00:00, 96.20it/s, Epoch=2, Val Loss=1.504022]
188it [00:06, 28.08it/s, Epoch=3, Loss=1.503548]
47it [00:00, 92.88it/s, Epoch=3, Val Loss=1.494690]
188it [00:06, 28.11it/s, Epoch=4, Loss=1.471231]
47it [00:00, 96.70it/s, Epoch=4, Val Loss=1.497066]
188it [00:06, 28.38it/s, Epoch=5, Loss=1.489780]
47it [00:00, 96.81it/s, Epoch=5, Val Loss=1.509968]
188it [00:06, 28.54it/s, Epoch=6, Loss=1.475149]
47it [00:00, 94.94it/s, Epoch=6, Val Loss=1.479062]
188it [00:06, 28.29it/s, Epoch=7, Loss=1.500863]
47it [00:00, 97.51it/s, Epoch=7, Val Loss=1.482664]
188it [00:06, 28.44it/s, Epoch=8, Loss=1.478105]
47it [00:00, 96.19it/s, Epoch=8, Val Loss=1.485463]
188it [00:06, 28.52it/s, Epoch=9, Loss=1.494310]
47it [00:00, 92.24it/s, Epoch=9, Val Loss=1.475710]
188it [00:06, 28.57it/s, Epoch=10, Loss=1.521073]
47it [00:00, 84.38it/s, Epoch=10, Val Los

Best: Epoch= 20  val_loss= 1.479166
